In [ ]:
import json
import numpy as np
import pandas as pd
from IPython.display import display
from ipywidgets import interact, widgets

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
with open("sets-definition.json") as f:
    sets_definition = json.load(f)

In [ ]:
bar_weight_definition = {
    "Olympic Ladies": 15.0,
    "Olympic Mens": 20.0,
    "Smith machine": 11.3,
}

bar_weight_dropdown = widgets.Dropdown(
    options=bar_weight_definition.keys(),
    value="Olympic Ladies",
    description="Bar weight:",
    disabled=False,
)

In [ ]:
working_weight_input = widgets.BoundedFloatText(
    value=0.0,
    min=0,
    max=200.0,
    step=0.1,
    description='Working weight:',
    disabled=False
)

In [ ]:
@interact
def calculate_warmup_sets(bar_weight=bar_weight_dropdown, working_weight=working_weight_input):
    bar_weight = bar_weight_definition[bar_weight]

    sets = pd.DataFrame({})

    for warmup_set in sets_definition:
        if warmup_set["set_num"] == 1:
            tmp_df = pd.DataFrame(
                {
                    "Set": 1,
                    "Reps": warmup_set["reps"],
                    "Weight(s)": "empty bar",
                    "Min. Weight": "",
                    "Max. Weight": "",
                    "Description": warmup_set["description"],
                },
                index=[0],
            )
    
        else:
            min_weight = working_weight * warmup_set["min_weight_multiplier"]
            max_weight = working_weight * warmup_set["max_weight_multiplier"]
            bar_within_range = (bar_weight >= min_weight) and (bar_weight <= max_weight)
            bar_above_range = bar_weight >= max_weight
    
            if bar_within_range or bar_above_range:
                continue
    
            weight_range = np.arange(np.floor(min_weight), np.ceil(max_weight) + 1)
            recc_weight = list(filter(lambda x: (x % 2.5 == 0), weight_range))
    
            if len(recc_weight) == 0:
                continue

            if str(recc_weight) in sets["Weight(s)"].values:
                continue
            
            tmp_df = pd.DataFrame(
                {
                    "Set": warmup_set["set_num"],
                    "Reps": warmup_set["reps"],
                    "Weight(s)": str(recc_weight),
                    "Min. Weight": min_weight,
                    "Max. Weight": max_weight,
                    "Description": warmup_set["description"],
                },
                index=[0],
            )
                
        sets = pd.concat([sets, tmp_df], ignore_index=True)
    
    if len(sets) < len(sets_definition):
        sets["Set"] = np.arange(1, len(sets) + 1)
    
    sets.set_index("Set", inplace=True)

    display(sets)